In [3]:
import bpy
import bmesh
from math import radians, ceil, floor, sqrt
from random import randint, uniform, sample, choice
import os
import numpy as np
import pychrono.core as chrono
import pychrono.irrlicht as chronoirr

chrono.SetChronoDataPath('C:/Users/sbrege/Anaconda3/pkgs/pychrono-6.0.0-py38_0/Library/data')
np.set_printoptions(threshold=sys.maxsize)

template = 'C:/Users/sbrege/Documents/gen/grid5.blend'
numVertices = 25
path = r"C:\Users\sbrege\Documents\gen\generations"
cupName = r"\cup"
ballName = r"\ball"
objExt = ".obj"
blendExt = ".blend"
xRange = np.arange(-0.225,0.226,0.05)
yRange = np.arange(-0.225,0.226,0.05)
moves = [-0.05,-1,0,1,2]

In [2]:
class Cup:
    def __init__(self, vertexMap = None, ID = None, objPath = None):
        self.vertexMap = vertexMap
        self.ID = ID
        self.objPath = objPath
    
    def fitness(self):
        mysystem = chrono.ChSystemNSC()
        chrono.ChCollisionModel.SetDefaultSuggestedEnvelope(0.001)
        chrono.ChCollisionModel.SetDefaultSuggestedMargin(0.001)
        contact_material = chrono.ChMaterialSurfaceNSC()
        body_A= chrono.ChBodyEasyMesh(self.objPath, # mesh filename
                                       7000,             # density kg/m^3
                                       True,             # automatically compute mass and inertia
                                       True,             # visualize?>
                                       True,             # collide?
                                       contact_material, # contact material
                                       )
        body_A.SetBodyFixed(True)
        mysystem.Add(body_A)

        body_B= chrono.ChBodyEasyMesh('C:/Users/sbrege/Documents/gen/generations/ball.obj', # mesh filename
                                     1000,             # density kg/m^3
                                     True,             # automatically compute mass and inertia
                                     True,             # visualize?>
                                     True,             # collide?
                                     contact_material, # contact material
                                     )
        body_B.SetPos(chrono.ChVectorD(0,1,0))
        mysystem.Add(body_B)

        distanceList = np.asarray([]) #Dats fitness bayBee!

        mysystem.SetChTime(0)
        while (mysystem.GetChTime() < 1.2) :
            mysystem.DoStepDynamics(0.0001)
            xVal = body_B.GetPos().x
            yVal = body_B.GetPos().y
            zVal = body_B.GetPos().z
            distance = sqrt(xVal**2 + yVal**2 + zVal**2)
            distanceList = np.append(distanceList, distance)

        return np.mean(distanceList)

In [6]:
def makeCup(ID, vertexMap):  
    bpy.ops.wm.open_mainfile(filepath=template)
    context = bpy.context

    bpy.ops.object.mode_set(mode = 'EDIT') 
    ob = context.object
    me = ob.data
    bm = bmesh.from_edit_mesh(me)
    i=0
    for v in bm.verts:
        if v.co.z > 0:
            v.select_set(True)
            bpy.ops.transform.translate(value=(0, 0, vertexMap[i]))
            v.select_set(False)
            i+=1
    bm.select_mode |= {'VERT'}
    bm.select_flush_mode()
    
    bmesh.update_edit_mesh(me)
    
    objPath = path + cupName + str(ID) + objExt

    bpy.ops.export_scene.obj(filepath=objPath)

    return Cup(vertexMap=vertexMap, ID=ID, objPath = objPath)


def randomCup(ID, minHeight = 0, maxHeight = 0.3):
    """
    ID: int, used for finding relevant '.obj' and '.blend' files
    minBricks: int, minimum number of bricks to place
    minBricks: int, maximum number of bricks to place
    This function creates a cup.
    """

    vertexMap = np.zeros(numVertices) #contains vertex height at each point

    for point in np.nditer(vertexMap, op_flags=['readwrite']):
        point[...] = uniform(minHeight,maxHeight)

    return makeCup(ID, vertexMap)

def createBall(size):
    """
    creates ball for fitness
    """
    bpy.ops.object.select_all(action='DESELECT')
    for o in bpy.context.scene.objects:
        o.select_set(True)
    bpy.ops.object.delete()
    bpy.ops.mesh.primitive_uv_sphere_add(scale = (size,size,size))

    finalPath = path + ballName + objExt

    bpy.ops.export_scene.obj(filepath=finalPath)

def createPopulation(population, ballSize):
    """
    population: int, number of cups to make
    initializes by creating a base population of random cups
    """
    createBall(ballSize)
    popList = []
    for i in range(population):
        cup = randomCup(i)
        popList.append(cup)
    return popList

def rankCups(cups):
    scoreArr = np.asarray([0,0])
    i = 0
    for cup in cups:
        scoreArr = np.vstack((scoreArr, [i, cup.fitness()]))
        i +=1
    scoreArr = np.delete(scoreArr,0,axis=0)
    return scoreArr[np.argsort(scoreArr[:,1])]

def selectCups(population, rankedCups, eliteSize):
    selected = []
    for i in range(eliteSize):
        sCup = population[int(rankedCups[i][0])]
        selected.append(makeCup(i, sCup.vertexMap))
    return selected

def breed(parent1, parent2, ID):

    p1vMap = parent1.vertexMap
    p2vMap = parent2.vertexMap
    cvMap = np.zeros_like(p1vMap)
    #c2vMap = np.zeros_like(p1vMap)

    i = 0
    for vert in cvMap:
        if uniform(0,1) < 0.5:
            cvMap[i] = p1vMap[i]
        else:
            cvMap[i] = p2vMap[i]
        i+=1

    return makeCup(ID, cvMap)

def breedPopulation(elitePool, popSize):
    newPop = elitePool
    i=len(elitePool)
    eLen = len(elitePool)
    u=0
    v=1
    while len(newPop) < popSize:
        child = breed(elitePool[u], elitePool[u+v], i)
        i+=1
        newPop.append(child)
        v+=1
        print(u+v)
        if (u+v) >= eLen:
            u+=1
            v=1
        if u == eLen:
            u=0
            v=1
    return newPop

def mutate(cup, ID, rate):
    vertexMap = cup.vertexMap
    for point in np.nditer(vertexMap, op_flags=['readwrite']):
        if uniform(0,1) < rate:
            height = point + uniform(-0.1,0.1)
            if height < 0:
                height = 0
            elif height > 0.3:
                height = 0.3
            point[...] = height

    return makeCup(ID, vertexMap)

def mutatePopulation(population, mutationRate, eliteSize):
    newPopulation = []
    i=0
    for pop in population:
        if i < eliteSize:
            newPopulation.append(pop)
        else:
            newPopulation.append(mutate(pop, pop.ID, mutationRate))
    return newPopulation

def nextGeneration(population, eliteSize, mutationRate):
    rankedCups = rankCups(population)
    selectionResults = selectCups(population, rankedCups, eliteSize)
    children = breedPopulation(selectionResults, len(population))
    nextGeneration = mutatePopulation(children, mutationRate, eliteSize)
    return nextGeneration

def geneticCups(popSize, eliteSize, mutationRate, generations, bSize = 0.2):
    population = createPopulation(popSize, bSize)

    for gen in range(generations):
        print(gen, '-----------------------------------------------------------------------------------------------------------------------------------------------')
        population = nextGeneration(population, eliteSize, mutationRate)
        
    ranked = rankCups(population)
    selected = selectCups(population, ranked, 1)
#Automating the Mind_
    return selected

In [7]:
huh = randomCup(1)

    (  0.0010 sec |   0.0000 sec) OBJ Export path: 'C:\\Users\\sbrege\\Documents\\gen\\generations\\cup1.obj'
          (  0.0040 sec |   0.0030 sec) Finished writing geometry of 'Cube'.
      (  0.0040 sec |   0.0030 sec) Finished exporting geometry, now exporting materials
      (  0.0050 sec |   0.0040 sec) OBJ Export Finished
Progress: 100.00%



In [8]:
%%time
god = geneticCups(16,4,0.2,300, 0.2)

Info: Deleted 1 object(s)
    (  0.0000 sec |   0.0000 sec) OBJ Export path: 'C:\\Users\\sbrege\\Documents\\gen\\generations\\ball.obj'
          (  0.0150 sec |   0.0150 sec) Finished writing geometry of 'Sphere'.
      (  0.0150 sec |   0.0150 sec) Finished exporting geometry, now exporting materials
      (  0.0160 sec |   0.0160 sec) OBJ Export Finished
Progress: 100.00%

    (  0.0000 sec |   0.0000 sec) OBJ Export path: 'C:\\Users\\sbrege\\Documents\\gen\\generations\\cup0.obj'
          (  0.0020 sec |   0.0009 sec) Finished writing geometry of 'Cube'.
      (  0.0030 sec |   0.0030 sec) Finished exporting geometry, now exporting materials
      (  0.0030 sec |   0.0030 sec) OBJ Export Finished
Progress: 100.00%

    (  0.0000 sec |   0.0000 sec) OBJ Export path: 'C:\\Users\\sbrege\\Documents\\gen\\generations\\cup1.obj'
          (  0.0020 sec |   0.0010 sec) Finished writing geometry of 'Cube'.
      (  0.0030 sec |   0.0030 sec) Finished exporting geometry, now exporting mate

KeyboardInterrupt: 

In [31]:
god[0].ID

0

In [7]:
#ball
bpy.ops.object.select_all(action='DESELECT')

for o in bpy.context.scene.objects:
    o.select_set(True)
bpy.ops.object.delete()

bpy.ops.mesh.primitive_uv_sphere_add(scale = (0.17,0.17,0.17))


finalPath = path + ballName + objExt
bpy.ops.export_scene.obj(filepath=finalPath)

Info: Deleted 1 object(s)
    (  0.0000 sec |   0.0000 sec) OBJ Export path: 'C:\\Users\\sbrege\\Documents\\gen\\generations\\ball.obj'
          (  0.0242 sec |   0.0232 sec) Finished writing geometry of 'Sphere'.
      (  0.0242 sec |   0.0232 sec) Finished exporting geometry, now exporting materials
      (  0.0242 sec |   0.0232 sec) OBJ Export Finished
Progress: 100.00%



{'FINISHED'}

In [ ]:
#SUBTRACTION CODE - DONT RUN
bpy.ops.mesh.primitive_cube_add(size=0.2, location=(0,0,0.05), scale=(5,5,1))
bpy.ops.mesh.primitive_cube_add(size=0.05, location=(0,0,0.1), scale=(1,1,1))
base = bpy.context.scene.objects['Cube']
brick = bpy.context.scene.objects['Cube.001']
bpy.ops.object.select_all(action='SELECT')
bpy.context.view_layer.objects.active = base

bpy.ops.object.make_single_user(object=True, obdata=True)
bpy.ops.object.convert(target='MESH')
bpy.ops.object.editmode_toggle()
bpy.ops.mesh.reveal()
bpy.ops.mesh.select_all(action='DESELECT')
bpy.ops.object.editmode_toggle()
bpy.ops.object.editmode_toggle()
bpy.ops.mesh.reveal()
bpy.ops.mesh.select_all(action='SELECT')
bpy.ops.object.editmode_toggle()
bpy.ops.object.modifier_apply(modifier="Auto Boolean")
bpy.ops.object.booltool_auto_difference()


In [41]:

mysystem = chrono.ChSystemNSC()
chrono.ChCollisionModel.SetDefaultSuggestedEnvelope(0.001)
chrono.ChCollisionModel.SetDefaultSuggestedMargin(0.001)
contact_material = chrono.ChMaterialSurfaceNSC()

brick_material = chrono.ChMaterialSurfaceNSC()
brick_material.SetFriction(0.5)
brick_material.SetDampingF(0.2)
brick_material.SetCompliance (0.0000001)
brick_material.SetComplianceT(0.0000001)
brick_material.SetRollingFriction(0)
brick_material.SetSpinningFriction(0)
brick_material.SetComplianceRolling(0.0000001)
brick_material.SetComplianceSpinning(0.0000001)

body_floor = chrono.ChBody()
body_floor.SetBodyFixed(True)
body_floor.SetPos(chrono.ChVectorD(0, -2, 0 ))

# Collision shape
body_floor.GetCollisionModel().ClearModel()
body_floor.GetCollisionModel().AddBox(brick_material, 3, 1, 3) # hemi sizes
body_floor.GetCollisionModel().BuildModel()
body_floor.SetCollide(True)

# Visualization shape
body_floor_shape = chrono.ChBoxShape()
body_floor_shape.GetBoxGeometry().Size = chrono.ChVectorD(3, 1, 3)
body_floor.GetAssets().push_back(body_floor_shape)

body_floor_texture = chrono.ChTexture()
body_floor_texture.SetTextureFilename('C:/Users/sbrege/Anaconda3/pkgs/pychrono-6.0.0-py38_0/Library/data/textures/concrete.jpg')
body_floor.GetAssets().push_back(body_floor_texture)

mysystem.Add(body_floor)

body_A= chrono.ChBodyEasyMesh('C:/Users/sbrege/Documents/gen/generations/cup0.obj', # mesh filename
                              7000,             # density kg/m^3
                              True,             # automatically compute mass and inertia
                              True,             # visualize?>
                              True,             # collide?
                              contact_material, # contact material
                              )
#body_A.SetPos(chrono.ChVectorD(0.5,0.5,0))
body_A.SetBodyFixed(True)
mysystem.Add(body_A)

body_B= chrono.ChBodyEasyMesh('C:/Users/sbrege/Documents/gen/generations/ball.obj', # mesh filename
                              1,             # density kg/m^3
                              True,             # automatically compute mass and inertia
                              True,             # visualize?>
                              True,             # collide?
                              contact_material, # contact material
                              )
body_B.SetPos(chrono.ChVectorD(0,1,0))
mysystem.Add(body_B)

myapplication = chronoirr.ChIrrApp(mysystem, 'PyChrono example', chronoirr.dimension2du(1024,768))

myapplication.AddTypicalSky('C:/Users/sbrege/Anaconda3/pkgs/pychrono-6.0.0-py38_0/Library/data/skybox/')
myapplication.AddTypicalCamera(chronoirr.vector3df(0.5,0.5,1), chronoirr.vector3df(0,0,0))
#myapplication.AddTypicalLights()
myapplication.AddLightWithShadow(chronoirr.vector3df(3,6,2),    # point
                                 chronoirr.vector3df(0,0,0),    # aimpoint
                                 12,                 # radius (power)
                                 1,11,              # near, far
                                 55)             
myapplication.AssetBindAll()
myapplication.AssetUpdateAll()
myapplication.AddShadowAll()
myapplication.SetTimestep(0.0001)

while(myapplication.GetDevice().run()):
    myapplication.BeginScene()
    myapplication.DrawAll()
    myapplication.DoStep()
    myapplication.EndScene()


In [20]:
#THIS ONE MOVES
mysystem = chrono.ChSystemNSC()
chrono.ChCollisionModel.SetDefaultSuggestedEnvelope(0.001)
chrono.ChCollisionModel.SetDefaultSuggestedMargin(0.001)
contact_material = chrono.ChMaterialSurfaceNSC()

brick_material = chrono.ChMaterialSurfaceNSC()
brick_material.SetFriction(0.2)
brick_material.SetDampingF(0.2)
brick_material.SetCompliance (0.0000001)
brick_material.SetComplianceT(0.0000001)
brick_material.SetRollingFriction(0)
brick_material.SetSpinningFriction(0)
brick_material.SetComplianceRolling(0.0000001)
brick_material.SetComplianceSpinning(0.0000001)

body_floor = chrono.ChBody()
body_floor.SetBodyFixed(True)
body_floor.SetPos(chrono.ChVectorD(0, -2, 0 ))

# Collision shape
body_floor.GetCollisionModel().ClearModel()
body_floor.GetCollisionModel().AddBox(brick_material, 3, 1, 3) # hemi sizes
body_floor.GetCollisionModel().BuildModel()
body_floor.SetCollide(True)

# Visualization shape
body_floor_shape = chrono.ChBoxShape()
body_floor_shape.GetBoxGeometry().Size = chrono.ChVectorD(3, 1, 3)
body_floor.GetAssets().push_back(body_floor_shape)

body_floor_texture = chrono.ChTexture()
body_floor_texture.SetTextureFilename('C:/Users/sbrege/Anaconda3/pkgs/pychrono-6.0.0-py38_0/Library/data/textures/concrete.jpg')
body_floor.GetAssets().push_back(body_floor_texture)

mysystem.Add(body_floor)

# body_A= chrono.ChBodyEasyMesh('C:/Users/sbrege/Documents/gen/generations/cup0.obj', # mesh filename
#                               7000,             # density kg/m^3
#                               True,             # automatically compute mass and inertia
#                               True,             # visualize?>
#                               True,             # collide?
#                               contact_material, # contact material
#                               )
# #body_A.SetPos(chrono.ChVectorD(0.5,0.5,0))
# body_A.SetBodyFixed(True)
# mysystem.Add(body_A)

body_B= chrono.ChBodyEasyMesh('C:/Users/sbrege/Documents/gen/generations/ball.obj', # mesh filename
                              1,             # density kg/m^3
                              True,             # automatically compute mass and inertia
                              True,             # visualize?>
                              True,             # collide?
                              contact_material, # contact material
                              )
body_B.SetPos(chrono.ChVectorD(0,1.2,0))
mysystem.Add(body_B)

body_table= chrono.ChBodyEasyMesh('C:/Users/sbrege/Documents/gen/idealCup.obj', # mesh filename
                              7000,             # density kg/m^3
                              True,             # automatically compute mass and inertia
                              True,             # visualize?>
                              True,             # collide?
                              brick_material, # contact material
                              )

mysystem.Add(body_table)


# Create a constraint that blocks free 3 x y z translations and 3 rx ry rz rotations
# of the table respect to the floor, and impose that the relative imposed position
# depends on a specified motion law.

link_shaker = chrono.ChLinkLockLock()
link_shaker.Initialize(body_table, body_floor, chrono.CSYSNORM)
mysystem.Add(link_shaker)

# ..create the function for imposed x horizontal motion, etc.
#mfunY = chrono.ChFunction_Sine(0,1.5,0.001)  # phase, frequency, amplitude
#link_shaker.SetMotion_Y(mfunY)

# ..create the function for imposed y vertical motion, etc.
mfunZ = chrono.ChFunction_Sine(0,0.6,0.12)  # phase, frequency, amplitude
link_shaker.SetMotion_Z(mfunZ)

mfunX = chrono.ChFunction_Sine(0,0.6,0.12)
link_shaker.SetMotion_X(mfunX)

myapplication = chronoirr.ChIrrApp(mysystem, 'PyChrono example', chronoirr.dimension2du(1024,768))

myapplication.AddTypicalSky('C:/Users/sbrege/Anaconda3/pkgs/pychrono-6.0.0-py38_0/Library/data/skybox/')
myapplication.AddTypicalCamera(chronoirr.vector3df(0.5,0.5,1), chronoirr.vector3df(0,0,0))
#myapplication.AddTypicalLights()
myapplication.AddLightWithShadow(chronoirr.vector3df(3,6,2),    # point
                                 chronoirr.vector3df(0,0,0),    # aimpoint
                                 12,                 # radius (power)
                                 1,11,              # near, far
                                 55)             
myapplication.AssetBindAll()
myapplication.AssetUpdateAll()
myapplication.AddShadowAll()
myapplication.SetTimestep(0.0001)

while(myapplication.GetDevice().run()):
    myapplication.BeginScene()
    myapplication.DrawAll()
    myapplication.DoStep()
    myapplication.EndScene()


In [207]:
xpoint

-0.014050833421274183

In [220]:
1/np.mean(distanceList)

2.7848558760454893

In [218]:
1/np.mean(distanceList)

1.3010575963526514